In [ ]:
!pip install --q git+https://github.com/m-bain/whisperx.git

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.6/208.6 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 630.6/630.6 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import whisperx
import gc

/usr/local/lib/python3.10/dist-packages/pyannote/audio/core/io.py:43: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")
/usr/local/lib/python3.10/dist-packages/torch_audiomentations/utils/io.py:27: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")


In [ ]:
device = "cuda"
batch_size = 4 # reduce if low on GPU mem
compute_type = "float16" # change to "int8" if low on GPU mem (may reduce accuracy)

In [ ]:
audio_file = "sam_altman_lex_podcast_367_short.wav"

In [ ]:
audio = whisperx.load_audio(audio_file)

In [ ]:
model = whisperx.load_model("large-v2", device, compute_type=compute_type)


No language specified, language will be first be detected for each audio file (increases inference time).


INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.1.2. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/torch/whisperx-vad-segmentation.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.1.0. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.1.0+cu118. Bad things might happen unless you revert torch to 1.x.


In [ ]:
result = model.transcribe(audio, batch_size=batch_size)
print(result["segments"]) # before alignment

# delete model if low on GPU resources
# import gc; gc.collect(); torch.cuda.empty_cache(); del model

# 2. Align whisper output
model_a, metadata = whisperx.load_align_model(language_code=result["language"], device=device)
result = whisperx.align(result["segments"], model_a, metadata, audio, device, return_char_alignments=False)


Detected language: en (1.00) in first 30s of audio...
[{'text': ' And RLHF is how we take some human feedback. The simplest version of this is show two outputs, ask which one is better than the other, which one the human raters prefer, and then feed that back into the model with reinforcement learning. And that process works remarkably well with, in my opinion, remarkably little data to make the model more useful. So RLHF is how we', 'start': 0.009, 'end': 24.735}, {'text': " Align the model to what humans wanted to do so there's a giant language model that strain a giant data set to create this kind of background wisdom knowledge is contained in the internet.", 'start': 24.735, 'end': 36.681}, {'text': " And then somehow adding a little bit of human guidance on top of it through this process makes it seem so much more awesome. Maybe just because it's much easier to use. It's much easier to get what you want. You get it right more often the first time. And ease of use matters a lot, ev

Downloading: "https://download.pytorch.org/torchaudio/models/wav2vec2_fairseq_base_ls960_asr_ls960.pth" to /root/.cache/torch/hub/checkpoints/wav2vec2_fairseq_base_ls960_asr_ls960.pth
100%|██████████| 360M/360M [00:06<00:00, 60.6MB/s]


In [ ]:
result

{'segments': [{'start': 0.309,
   'end': 3.39,
   'text': ' And RLHF is how we take some human feedback.',
   'words': [{'word': 'And', 'start': 0.309, 'end': 0.389, 'score': 0.985},
    {'word': 'RLHF', 'start': 0.569, 'end': 1.149, 'score': 0.652},
    {'word': 'is', 'start': 1.329, 'end': 1.409, 'score': 0.508},
    {'word': 'how', 'start': 1.469, 'end': 1.729, 'score': 0.845},
    {'word': 'we', 'start': 1.929, 'end': 2.09, 'score': 0.869},
    {'word': 'take', 'start': 2.17, 'end': 2.41, 'score': 0.833},
    {'word': 'some', 'start': 2.45, 'end': 2.61, 'score': 0.752},
    {'word': 'human', 'start': 2.65, 'end': 2.91, 'score': 0.912},
    {'word': 'feedback.', 'start': 2.95, 'end': 3.39, 'score': 0.808}]},
  {'start': 3.97,
   'end': 14.432,
   'text': 'The simplest version of this is show two outputs, ask which one is better than the other, which one the human raters prefer, and then feed that back into the model with reinforcement learning.',
   'words': [{'word': 'The', 'start'

In [ ]:
diarize_model = whisperx.DiarizationPipeline(use_auth_token="HUGGINGFACE_TOKEN",
                                             device=device)

In [ ]:
diarize_segments = diarize_model(audio, min_speakers=2, max_speakers=2)

In [ ]:
diarize_segments

,segment,label,speaker,start,end
0,[ 00:00:00.008 --> 00:00:00.025],A,SPEAKER_01,0.008489,0.025467
1,[ 00:00:00.195 --> 00:00:36.697],B,SPEAKER_01,0.195246,36.697793
2,[ 00:00:37.835 --> 00:00:38.327],C,SPEAKER_01,37.835314,38.327674
3,[ 00:00:39.159 --> 00:00:44.643],D,SPEAKER_01,39.159593,44.643463
4,[ 00:00:45.475 --> 00:00:48.022],E,SPEAKER_01,45.475382,48.022071
...,...,...,...,...,...
83,[ 00:07:39.142 --> 00:07:45.526],CF,SPEAKER_01,459.142615,465.526316
84,[ 00:07:46.986 --> 00:07:50.161],CG,SPEAKER_01,466.986418,470.161290
85,[ 00:07:50.738 --> 00:07:51.061],CH,SPEAKER_00,470.738540,471.061121
86,[ 00:07:51.061 --> 00:07:53.471],CI,SPEAKER_01,471.061121,473.471986


In [ ]:
diarize_segments.speaker.unique()

array(['SPEAKER_01', 'SPEAKER_00'], dtype=object)

In [ ]:
result = whisperx.assign_word_speakers(diarize_segments, result)
print(diarize_segments)
print(result["segments"]) # segments are now assigned speaker IDs

                              segment label     speaker       start  \
0   [ 00:00:00.008 -->  00:00:00.025]     A  SPEAKER_01    0.008489   
1   [ 00:00:00.195 -->  00:00:36.697]     B  SPEAKER_01    0.195246   
2   [ 00:00:37.835 -->  00:00:38.327]     C  SPEAKER_01   37.835314   
3   [ 00:00:39.159 -->  00:00:44.643]     D  SPEAKER_01   39.159593   
4   [ 00:00:45.475 -->  00:00:48.022]     E  SPEAKER_01   45.475382   
..                                ...   ...         ...         ...   
83  [ 00:07:39.142 -->  00:07:45.526]    CF  SPEAKER_01  459.142615   
84  [ 00:07:46.986 -->  00:07:50.161]    CG  SPEAKER_01  466.986418   
85  [ 00:07:50.738 -->  00:07:51.061]    CH  SPEAKER_00  470.738540   
86  [ 00:07:51.061 -->  00:07:53.471]    CI  SPEAKER_01  471.061121   
87  [ 00:07:53.777 -->  00:07:58.463]    CJ  SPEAKER_01  473.777589   

           end  intersection       union  
0     0.025467   -478.264533  478.361511  
1    36.697793   -441.592207  478.174754  
2    38.327674   -

In [ ]:
result

{'segments': [{'start': 0.309,
   'end': 3.39,
   'text': ' And RLHF is how we take some human feedback.',
   'words': [{'word': 'And',
     'start': 0.309,
     'end': 0.389,
     'score': 0.985,
     'speaker': 'SPEAKER_01'},
    {'word': 'RLHF',
     'start': 0.569,
     'end': 1.149,
     'score': 0.652,
     'speaker': 'SPEAKER_01'},
    {'word': 'is',
     'start': 1.329,
     'end': 1.409,
     'score': 0.508,
     'speaker': 'SPEAKER_01'},
    {'word': 'how',
     'start': 1.469,
     'end': 1.729,
     'score': 0.845,
     'speaker': 'SPEAKER_01'},
    {'word': 'we',
     'start': 1.929,
     'end': 2.09,
     'score': 0.869,
     'speaker': 'SPEAKER_01'},
    {'word': 'take',
     'start': 2.17,
     'end': 2.41,
     'score': 0.833,
     'speaker': 'SPEAKER_01'},
    {'word': 'some',
     'start': 2.45,
     'end': 2.61,
     'score': 0.752,
     'speaker': 'SPEAKER_01'},
    {'word': 'human',
     'start': 2.65,
     'end': 2.91,
     'score': 0.912,
     'speaker': 'SPEAKE